### Задание 1

Cпарсить страницу со свежеми новостям на habr.com/ru/all/.

Необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяются в начале кода в переменной, например: KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime as dt
from datetime import timedelta
import re

In [29]:
# Ключевые слова для поиска

KEYWORDS = ['python', 'парсинг']

In [30]:
# Отправка get-запроса

URL = 'https://habr.com/ru/all/'
req = requests.get(URL)

In [31]:
# Создание объекта BeautifulSoup

soup = BeautifulSoup(req.text, 'html.parser')
articles = soup.find_all('article', class_='post')

In [37]:
# Создание пустого датафрейма
hubs = pd.DataFrame()

for article in articles: 
    article_low = article.text.lower()    # Извлечение текста превью и перевод в нижний регистр
    if any([word in article_low for word in KEYWORDS]):    # Поиск ключевых слов в тексте превью
        date_ = article.find('span', class_ = 'post__time').text    # Дата публикации
        if 'сегодня' in date_.lower():
            date = dt.datetime.now().date()
        elif 'вчера' in date_.lower():
            date = dt.datetime.now().date() - timedelta(days = 1)
        else:
            date = date_
        title = article.find('a', class_ = 'post__title_link').text    # Заголовок статьи
        link = article.find('a', class_ = 'post__title_link').get('href')  # Ссылка 
        hub = {'дата': date, 'заголовок': title, 'ссылка': link}
        hubs = pd.concat([hubs, pd.DataFrame([hub])])
        
hubs

""


In [38]:
# Получение основной информации + текст поста

full_post = pd.DataFrame()

for article in articles:
    
    links = article.find('a', class_ = 'post__title_link').get('href')  # Ссылка
#     print(links)

    # Собираем заголовок, тэги и текст поста со страницы
    req_1 = requests.get(links)
    soup_1 = BeautifulSoup(req_1.text, 'html.parser')
    articles_1 = soup_1.find_all('article', class_='post')
    for art in articles_1:
        info = art.text.lower()
        if any([word in info for word in KEYWORDS]):
            date_ = art.find('span', class_ = 'post__time').text    # Дата публикации
            if 'сегодня' in date_.lower():
                date = dt.datetime.now().date()
            elif 'вчера' in date_.lower():
                date = dt.datetime.now().date() - timedelta(days = 1)
            else:
                date = date_
            title = art.find('span', class_ = 'post__title-text').text    # Заголовок статьи
            text = art.find('div', class_ = 'post__text').text            # Ссылка 
            post = {'дата': date, 'заголовок': title, 'ссылка': links, 'текст_статьи': text}
            full_post = pd.concat([full_post, pd.DataFrame([post])])
        
full_post.head(10)    

,дата,заголовок,ссылка,текст_статьи
0,2021-04-26,Операционализация: наука создания показателей ...,https://habr.com/ru/company/skillfactory/blog/...,"Пришло время затронуть психологическую тему, к..."
0,2021-04-26,Facebook отвлекает от работы? Блокируем его пр...,https://habr.com/ru/company/selectel/blog/554256/,\r\nПрокрастинация хорошо знакома каждому из н...
0,2021-04-26,Как устроен портативный автоматический ручной ...,https://habr.com/ru/company/skillfactory/blog/...,Будущее здесь — безо всяких преувеличений. В н...


### Задание 2

Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [ ]:
# Передача адресов в цикле

In [39]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

EMAIL = ["xxx@x.ru", "yyy@y.com"]

headers = {'Vaar-Header-App-Build-Version': '1.0.0', 
           'Vaar-Header-App-Product': 'hackcheck-web-avast',
           'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
           'Vaar-Version': '0'}

In [40]:
def mail_hack_check (url, addresses):
        
    all_breaches = pd.DataFrame()     # Создание пустого датафрейма
    for mail in EMAIL:
        data = {'emailAddresses': [mail]}     # Полезная нагрузка запроса
        request = requests.post(URL, headers = headers, json = data)      # Отпарвка запроса     
        response =   request.text
        info = response.split('}}')
        for el in info:
            publish_date = re.findall(r'(?<="publishDate":").{20}(?=")', el)    # Дата утечки
            site = re.findall(r'(?<="site":")\S+(?=",)', el)                    # Источник утечки
            description = re.findall(r'(?<="description":").+(?=\.\\r|\.\"|\.\s+\"|\.\\n\))', el)    # Описание утечки
            if publish_date and site and description:
                breach = {'почта': mail, 'дата утечки': publish_date, 'источник утечки': site, 'описание утечки': description}
                all_breaches = pd.concat([all_breaches, pd.DataFrame([breach])])
    return all_breaches

In [41]:
mail_hack_check(URL, EMAIL)

,почта,дата утечки,источник утечки,описание утечки
0,xxx@x.ru,[2017-02-14T00:00:00Z],[parapa.mail.ru],"[In July and August 2016, two criminals execut..."
0,xxx@x.ru,[2016-10-29T00:00:00Z],[vk.com],[Popular Russian social networking platform VK...
0,xxx@x.ru,[2016-10-21T00:00:00Z],[adobe.com],"[In October of 2013, criminals penetrated Adob..."
0,xxx@x.ru,[2017-02-14T00:00:00Z],[cfire.mail.ru],"[In July and August of 2016, two criminals car..."
0,xxx@x.ru,[2017-01-31T00:00:00Z],[cdprojektred.com],"[In March 2016, CDProjektRed.com.com's forum d..."
0,xxx@x.ru,[2016-10-23T00:00:00Z],[imesh.com],"[In June 2016, a cache of over 51 million user..."
0,yyy@y.com,[2019-03-28T00:00:00Z],[verifications.io],[Big data e-mail verification platform verific...
0,yyy@y.com,[2019-02-21T00:00:00Z],[www.dangdang.com],"[This is a list of email addresses only, and a..."
0,yyy@y.com,[2020-11-19T00:00:00Z],[123rf.com],"[In March 2020, the stock image agency 123RF w..."
0,yyy@y.com,[2020-01-03T00:00:00Z],[azcentral.com],"[At an unconfirmed date, online Arizona newspa..."


In [ ]:
# Передача одного адреса

In [42]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

mails = {'emailAddresses': ['xxx@x.com']}

headers = {'Vaar-Header-App-Build-Version': '1.0.0', 
           'Vaar-Header-App-Product': 'hackcheck-web-avast',
           'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
           'Vaar-Version': '0'}

In [43]:
# Отправка запроса

request__ = requests.post(URL, headers = headers, json = mails)

In [44]:
# Обработка ответа
response = request__.text
info = response.split('}}')

all_breaches_1 = pd.DataFrame()

for el in info:
    publish_date = re.findall(r'(?<="publishDate":").{20}(?=")', el)   # Дата утечки
    site = re.findall(r'(?<="site":")\S+(?=",)', el)                    # Источник утечки
    description = re.findall(r'(?<="description":").+(?=\.\\r|\.\"|\.\s+\"|\.\\n\))', el)       # Описание утечки
    if publish_date or site or description:
        breach = {'почта': mails['emailAddresses'], 'дата утечки': publish_date, 'источник утечки': site, 'описание утечки': description}
        all_breaches_1 = pd.concat([all_breaches_1, pd.DataFrame([breach])])
    
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
all_breaches_1

,почта,дата утечки,источник утечки,описание утечки
0,[xxx@x.com],[2016-11-07T00:00:00Z],[ir.netease.com],"[In October 2015, the Chinese internet and gam..."
0,[xxx@x.com],[2019-07-25T00:00:00Z],[socialflight.com],"[At an unconfirmed date, pilot app Social Flig..."
0,[xxx@x.com],[2017-09-07T00:00:00Z],[hiapk.com],[Chinese technology site HIAPK.com's user data...
0,[xxx@x.com],[2017-06-12T00:00:00Z],[bolt.cd],"[In January 2016, Bolt.cd's user database was ..."
0,[xxx@x.com],[2017-02-14T00:00:00Z],[plu.com.cn],[When the Chinese gaming site Plu's user datab...
0,[xxx@x.com],[2019-03-07T00:00:00Z],[dubsmash.com],"[In December 2018, Dubsmash's database was all..."
0,[xxx@x.com],[2019-10-03T00:00:00Z],[alora.io],"[In January 2018, runescape server Alora's dat..."
0,[xxx@x.com],[2017-04-14T00:00:00Z],[lupaworld.com],"[In November 2016, LupaWorld's user database w..."
0,[xxx@x.com],[2017-03-31T00:00:00Z],[tianya.cn],"[In 2011, Tianya's user database was allegedly..."
0,[xxx@x.com],[2019-10-24T00:00:00Z],[btcmonk],"[In September 2019, the online bitcoin trading..."


In [45]:
all_breaches_1.shape

(89, 4)